In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from sklearn.metrics import confusion_matrix, accuracy_score
import skopt
import scipy
from skopt.space import Real
import category_encoders
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import ComplementNB
import math

PREDICCION_REAL = False
MAXIMIZAR_HIPERPARAMETROS = False
PARAMETROS = {
    
}



In [2]:
#APERTURA DE ARCHIVO DE ARCHIVOS
entrenamiento = pd.read_pickle("../Archivos/Entrenamiento_Grupo_0.pkl")
test = pd.read_pickle("../Archivos/Validacion_Grupo_0.pkl").drop(columns = ["Opportunity_ID"])
test_real = pd.read_pickle("../Archivos/Validacion_Grupo_0.pkl")
if PREDICCION_REAL:
    test = pd.read_pickle("../Archivos/Test_Grupo_0.pkl").drop(columns = ["Opportunity_ID"])
    test_real = pd.read_pickle("../Archivos/Test_Grupo_0.pkl")

In [3]:
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12140 entries, 0 to 12139
Data columns (total 52 columns):
 #   Column                                                                 Non-Null Count  Dtype         
---  ------                                                                 --------------  -----         
 0   Opportunity_Name_Planned_Opportunity_Duration_mean                     12140 non-null  float64       
 1   Opportunity_Name_Planned_Opportunity_Duration_min                      12140 non-null  float64       
 2   Opportunity_Name_Planned_Time_Until_Deliver_mean                       12140 non-null  float64       
 3   Product_Name_Planned_Opportunity_Duration_mean                         12140 non-null  float64       
 4   Product_Name_Planned_Time_Until_Deliver_median                         12140 non-null  float64       
 5   Account_Name_Planned_Opportunity_Duration_median                       12140 non-null  float64       
 6   Account_Name_Planned_Time_Unti

In [4]:
#FILTRADO DE COLUMNAS - NO REMOVER STAGE O FECHA
#fugas = ['ID','Opportunity_Name','Sales_Contract_No','Account_Name','Account_Owner','Opportunity_Owner','Last_Modified_By','Product_Family','Product_Name','ASP','ASP_(converted)']
#'Total_Taxable_Amount'
#columnas_fecha = ['Month','Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
#entrenamiento = entrenamiento.drop(columns=fugas)
#test = test.drop(columns=fugas)
#test = test.drop(columns=columnas_fecha)
#entrenamiento = entrenamiento.drop(columns=columnas_fecha)

In [5]:
#FECHAS A DIAS

columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date',"Year-Month"]
def fecha_a_dias(x):
    for columna in columnas_fecha:
        x[columna] = x[columna].apply(lambda x : (pd.to_datetime(x) - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days)

fecha_a_dias(entrenamiento)
fecha_a_dias(test)

KeyError: 'Last_Modified_Date'

In [ ]:
objetivo = (entrenamiento['Stage'] == 1).astype(int)
entrenamiento = entrenamiento.drop(columns=['Stage'])
columnas_category = list(entrenamiento.select_dtypes(include=['category']).columns)
if 'Stage' in columnas_category : columnas_category.remove('Stage')

In [ ]:
if not PREDICCION_REAL:
    test_label = (test['Stage'] == 1).astype(int)
    test = test.drop(columns=['Stage'])

In [ ]:
def categoricas_a_numericas(train,test,label,usar_label):
    if (usar_label):
        columnas_object = list(train.select_dtypes(include=['category']).columns)
    else:
        columnas_object = list(test.select_dtypes(include=['category']).columns)
    if 'Stage' in columnas_object : columnas_object.remove('Stage')
    ohe = category_encoders.cat_boost.CatBoostEncoder(cols = columnas_object,return_df = True)
    ohe.fit(train,label)
    if (usar_label):
        columnas = ohe.transform(train,label)
        for columna in columnas_object:
            train[columna] = columnas[columna].copy()
    else:
        columnas = ohe.transform(test)
        for columna in columnas_object:
            test[columna] = columnas[columna].copy()
categoricas_a_numericas(entrenamiento,test,objetivo,False)
categoricas_a_numericas(entrenamiento,test,objetivo,True)

In [ ]:
def limpiar_nan(col):
    mean = entrenamiento[col].mean()
    entrenamiento[col] = entrenamiento[col].replace(np.NaN,mean)
    mean = test[col].mean()
    test[col] = test[col].replace(np.NaN,mean)

In [ ]:
def limpiar_inf(col):
    entrenamiento[col] = entrenamiento[col].replace(math.inf,np.NaN)
    test[col] = test[col].replace(math.inf,np.NaN)

In [ ]:
#entrenamiento = entrenamiento.drop(columns = ["Quote_Expiry_Date","Total_Products_Region_Quarter_Change"])
#test = test.drop(columns = ["Quote_Expiry_Date","Total_Products_Region_Quarter_Change"])

In [ ]:
for col in entrenamiento.columns:
    ent_null = entrenamiento.loc[entrenamiento[col] == np.inf]
    if (ent_null[col].count() > 1000):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 1000):
        print(col)
        print(ent_null[col].count())
        limpiar_inf(col)        
for col in test.columns:
    ent_null = test.loc[test[col] == np.inf]
    if (ent_null[col].count() > 1000):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 1000):
        print(col)
        print(ent_null[col].count())
        limpiar_inf(col)        

In [ ]:
for col in entrenamiento.columns:
    ent_null = entrenamiento.loc[entrenamiento[col].isnull()]
    ent_null = ent_null.replace(np.NaN,0)
    if (ent_null[col].count() > 1000):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 1000):
        print(col)
        print(ent_null[col].count())
        limpiar_nan(col)        
for col in test.columns:
    ent_null = test.loc[test[col].isnull()]
    ent_null = ent_null.replace(np.NaN,0)
    if (ent_null[col].count() > 1000):
        print(col)
        print(ent_null[col].count())
        entrenamiento = entrenamiento.drop(columns = [col])
        test = test.drop(columns = [col])
    if (0<ent_null[col].count() <= 1000):
        print(col)
        print(ent_null[col].count())
        limpiar_nan(col)        

In [ ]:
a_usar = ['Opportunity_Name_Planned_Opportunity_Duration_mean',
               'Opportunity_Name_Planned_Opportunity_Duration_min',
               'Opportunity_Name_Planned_Time_Until_Deliver_mean',
               'Product_Name_Planned_Opportunity_Duration_mean',
               'Product_Name_Planned_Time_Until_Deliver_median',
               'Account_Name_Planned_Opportunity_Duration_median',
               'Account_Name_Planned_Time_Until_Deliver_mean',
               'Product_Name_Planned_Time_Until_Deliver_std',
               'Product_Name_Product_Amount_Deviation_of_Product_Family_rate_mean',
               'Opportunity_Name_Total_Amount(USD)_mean',
               'Account_Name_Product_Amount_Deviation_of_Product_Family_rate_median',
               'Product_Name_Total_Amount(USD)_mean',
               'Opportunity_Name_Product_Amount_Deviation_of_Product_Family_rate_mean',
               'Bureaucratic_Code_Total_Taxable_Amount_max',
               'Bureaucratic_Code_Delivery_Year_mean',
               'Bureaucratic_Code_Total_Amount_by_Product_Family_mean_std',
               'Bureaucratic_Code_Total_Products_Region_This_Quarter_median',
               'Bureaucratic_Code_Total_Product_Family_Region_Month_Change_mean',
               'Last_Modified_By_Opportunity_Total_Amount_Region_std_Ratio_max',
               'Bureaucratic_Code_Pricing, Delivery_Terms_Approved_std',
               'Bureaucratic_Code_Bureaucratic_Code_0_Approved_mean',
               'Product_Name_Planned_Deliver_Duration_std',
               'Product_Name_Total_Taxable_Amount(USD)_std',
               'Account_Name_Total_Amount(USD)_mean',
               'Bureaucratic_Code_Total_Products_Region_Last_Week_mean',
               'Bureaucratic_Code_Opportunity_TRF_std',
               'Product_Name_Total_Amount_median',
               'Bureaucratic_Code_ASP_(converted)_max',
               'Bureaucratic_Code_Total_Taxable_Amount(USD)_std',
               'Bureaucratic_Code_Opportunity_TRF_Region_avg_Ratio_std',
               'Account_Name_Planned_Opportunity_Duration_min',
               'Product_Name_Planned_Deliver_Duration_median',
               'Last_Modified_By_Opportunity_ID_mean',
               'Bureaucratic_Code_Total_Products_Region_Last_Quarter_std',
               'Bureaucratic_Code_Planned_Opportunity_Duration_mean',
               'Product_Name_Total_Taxable_Amount_mean',
               'Bureaucratic_Code_Total_Products_Region_This_Week_median',
               'Account_Name_Actual_Opportunity_Duration_min',
               'Product_Name_Opportunity_TRF_Region_std_Ratio_std',
               'Bureaucratic_Code_Total_Amount_by_Billing_Country_std_mean',
               'Product_Name_Opportunity_TRF_Region_avg_Ratio_mean',
               'Account_Name_Total_Amount_by_Product_Family_mean_mean',
               'Product_Name_TRF_mean',
               'Account_Name_Product_Amount_Deviation_of_Product_Family_rate_min',
               'Last_Modified_By_Opportunity_TRF_Region_std_Ratio_max',
               'Product_Name_Opportunity_TRF_std',
               'Account_Name_Month_std',
               'Product_Name_Opportunity_Duration_by_Account_Type_std',
               'Bureaucratic_Code_Pricing, Delivery_Terms_Approved_median',
               'Bureaucratic_Code_Month_median']

drop = list(entrenamiento.columns)
for col in a_usar:
    drop.remove(col)
    
entrenamiento = entrenamiento.drop(columns = drop)

In [ ]:
test = test.drop(columns = drop)

In [ ]:
entrenamiento.info()

In [ ]:
model = GaussianNB(priors = [0.4,0.6],var_smoothing =0.01)

model.fit(entrenamiento,objetivo)
# make the prediction using the resulting model
preds = model.predict_proba(test)
preds

In [ ]:
if not PREDICCION_REAL:
    #accuracy_score(test_label, [1])
    print(skl.metrics.log_loss(test_label,[p[1] for p in preds]))

In [ ]:
model.get_params()

In [ ]:
preds = model.predict(entrenamiento)
accuracy_score(objetivo, preds)

In [ ]:
if (PREDICCION_REAL):
    preds = model.predict_proba(test)
    resultados = test_real[['Opportunity_ID']].copy()
    resultados['Target'] = pd.Series([p[1] for p in preds])
    resultados = resultados.groupby('Opportunity_ID').mean()
    resultados = resultados.reset_index()
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
    resultados.to_csv("prediccion_naive_bayes.csv", index=False)
    resultados['Target'].value_counts()

if not PREDICCION_REAL:
    preds = model.predict_proba(test)
    resultados = test_real[['Opportunity_ID']].copy()
    print(resultados.count())
    resultados['Target'] = pd.Series([p[1] for p in preds])
    #resultados = resultados.groupby('Opportunity_ID').mean()
    #resultados = resultados.reset_index()
    print(resultados.count())
    #resultados['Target'] = resultados['Target'].apply(lambda x: int(x >= 0.5))    
    resultados.to_csv("prediccion_naive_bayes_val.csv", index=False)
    resultados['Target'].value_counts()

In [ ]:
if(not MAXIMIZAR_HIPERPARAMETROS): sys.exit()

In [ ]:
from itertools import combinations

In [ ]:
entrenamiento.info()

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold,GridSearchCV

# Classifier
bayes_cv_tuner = GridSearchCV(
estimator = skl.naive_bayes.GaussianNB(),
param_grid = {
        "var_smoothing" : [i/10000 for i in range(1,1000)],
        "priors" : [[i/10,1-i/10] for i in range(10)]
},
cv = skl.model_selection.TimeSeriesSplit(),
n_jobs = 50,
verbose = 1,
refit = True,
scoring = "neg_log_loss"
)

In [ ]:
def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    

    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [ ]:
from skopt.space import Real
from sklearn.model_selection import StratifiedKFold,GridSearchCV
columnas = ['Bureaucratic_Code',
 'Account_Name',
 'Opportunity_Name',
 'Opportunity_Owner',
 'Last_Modified_By',
 'Product_Family',
 'Product_Name',
 'Opportunity_Name_Planned_Time_Until_Deliver_mean',
 'Opportunity_Name_Planned_Opportunity_Duration_mean',
 'Product_Name_Planned_Time_Until_Deliver_mean',
 'Product_Name_Planned_Opportunity_Duration_mean',
 'Territory',
 'Account_Owner',
 'Account_Type',
 'Opportunity_Type',
 'Planned_Opportunity_Duration',
 'Bureaucratic_Code_ASP_(converted)_std',
 'Bureaucratic_Code_Total_Amount_mean',
 'Bureaucratic_Code_Total_Product_Family_Region_Last_Month_mean',
 'Bureaucratic_Code_Total_Product_Family_Region_Last_Month_std',
 'Bureaucratic_Code_Total_Products_Region_Last_Week_std',
 'Bureaucratic_Code_Total_Products_Region_Last_Month_mean',
 'Bureaucratic_Code_Planned_Deliver_Duration_mean',
 'Bureaucratic_Code_Planned_Deliver_Duration_std',
 'Bureaucratic_Code_Actual_Opportunity_Duration_std',
 'Bureaucratic_Code_Planned_Time_Until_Deliver_mean',
 'Bureaucratic_Code_Planned_Time_Until_Deliver_std',
 'Bureaucratic_Code_Planned_Opportunity_Duration_mean',
 'Bureaucratic_Code_Product_Amount_Deviation_of_Product_Family_rate_std',
 'Bureaucratic_Code_Opportunity_Duration_Ratio_std',
 'Bureaucratic_Code_Opportunity_Total_Amount_Region_avg_std',
 'Bureaucratic_Code_Opportunity_TRF_Region_std_Ratio_std',
 'Bureaucratic_Code_Opportunity_Duration_by_Account_Type_std',
 'Bureaucratic_Code_ASP_by_Billing_Country_mean_std',
 'Bureaucratic_Code_Opportunity_Duration_by_Product_Family_mean_mean',
 'Bureaucratic_Code_Buro_Approved_by_Product_Family_std',
 'Account_Name_Planned_Opportunity_Duration_mean',
 'Opportunity_Type_Planned_Time_Until_Deliver_mean',
 'Opportunity_Type_Opportunity_Duration_by_Account_Type_mean',
 'Last_Modified_By_Planned_Time_Until_Deliver_mean']
mejor_score = 10000
mejor_col = list(entrenamiento.columns)
mejor_param = ""
j = 0
for i in columnas[::-1]:
    bayes_cv_tuner = GridSearchCV(
    estimator = skl.naive_bayes.GaussianNB(),
    param_grid = {
            "var_smoothing" : [i/100 for i in range(1,1000)],
            "priors" : [[i/10,1-i/10] for i in range(10)]
    },
    cv = skl.model_selection.TimeSeriesSplit(),
    n_jobs = 50,
    verbose = 1,
    refit = True
    )   
    if (j < len(columnas) - 1):
        print(i)
        entrenamiento = entrenamiento.drop(columns = [i])
        test = test.drop(columns = [i])
    resultCAT = bayes_cv_tuner.fit(entrenamiento, objetivo)
    print(bayes_cv_tuner.best_score_)
    print(list(entrenamiento.columns))
    if (bayes_cv_tuner.best_score_ < mejor_score):
        print(j)
        mejor_score = bayes_cv_tuner.best_score_
        mejor_param = bayes_cv_tuner.best_params_
        mejor_col = list(entrenamiento.columns)
    j+=1    

In [ ]:
print(mejor_col)
print(mejor_score)
print(mejor_param)

In [ ]:
resultCAT = bayes_cv_tuner.fit(entrenamiento, objetivo)

In [ ]:
print(bayes_cv_tuner.best_score_)
print(bayes_cv_tuner.best_params_)

In [ ]:
preds = bayes_cv_tuner.predict_proba(test.values)

In [ ]:
skl.metrics.log_loss(test_label,[p[1] for p in preds])

In [ ]:
res = pd.DataFrame(bayes_cv_tuner.cv_results_)
res["mean_test_score"] = -res["mean_test_score"]

In [ ]:
res["param_priors_"] = res["param_priors"].transform(lambda x : x[0])

In [ ]:
res_var = res.loc[res["param_priors_"] == 0.4]

In [ ]:
from matplotlib import style
plt.plot(res_var.param_var_smoothing.values,res_var.mean_test_score.values)
plt.title("Evolución del Log-Loss variando Var_Smoothing")
plt.xlabel("Var_Smoothing")
plt.ylabel("Log-Loss")
style.use("seaborn")
plt.show()

In [ ]:
res_priors = res.loc[res["param_var_smoothing"] == 0.0001]

In [ ]:
res_priors["param_priors_"] = res_priors["param_priors"].transform(lambda x : x[0])

In [ ]:
res_priors = res_priors.loc[res_priors.param_priors_ >= 0.1]
plt.plot(res_priors.param_priors_.values,res_priors.mean_test_score.values)
plt.title("Evolución del Log-Loss variando Priors")
plt.xlabel("Priors")
plt.ylabel("Log-Loss")
plt.show()